In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from torch import optim
import os
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

from model.classes.veichle_detection_cnn import *

class CustomDataset(Dataset):
    def __init__(self, target1_dir, target0_dir, transform=None):
        """
        target1_dir: directory with images for target=1
        target0_dir: directory with images for target=0
        """
        self.target1_images = [os.path.join(target1_dir, f) for f in os.listdir(target1_dir)]
        self.target0_images = [os.path.join(target0_dir, f) for f in os.listdir(target0_dir)]
        self.images = self.target1_images + self.target0_images
        self.labels = [1] * len(self.target1_images) + [0] * len(self.target0_images)
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert("RGB")  # Load the image and convert it to RGB
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


# Folders containing the images
target1_dir = './data/target1/'  # Replace with the path to the target1 folder
target0_dir = './data/target0/'  # Replace with the path to the target0 folder

# Create the custom dataset
dataset = CustomDataset(target1_dir, target0_dir, transform=transform)

# Split the dataset into training (90%) and validation (10%)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the model
model = VehicleDetectionCNN()

# Set the loss function and optimizer
criterion = torch.nn.BCELoss()  # Binary Cross Entropy for a binary classification problem
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  # Labels in float format for BCELoss
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch {epoch+1}, Training Loss: {running_loss / len(train_loader)}')

        # Evaluate the model on the validation data
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                predicted = (outputs.squeeze() > 0.5).float()  # Use 0.5 threshold for classification
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=8)


Epoca 1, Loss Addestramento: 0.37135235113757
Accuratezza Validazione: 83.33%
Epoca 2, Loss Addestramento: 0.11446536212627377
Accuratezza Validazione: 100.00%
Epoca 3, Loss Addestramento: 0.0575568408572248
Accuratezza Validazione: 95.83%
Epoca 4, Loss Addestramento: 0.024361980534324954
Accuratezza Validazione: 97.92%
Epoca 5, Loss Addestramento: 0.005580993157179494
Accuratezza Validazione: 97.92%
Epoca 6, Loss Addestramento: 0.005193545469540238
Accuratezza Validazione: 97.92%
Epoca 7, Loss Addestramento: 0.0148508017664426
Accuratezza Validazione: 100.00%
Epoca 8, Loss Addestramento: 0.001403005315556324
Accuratezza Validazione: 100.00%


In [3]:
# train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

In [4]:
torch.save(model.state_dict(), './model/saved/park_model.pth')